In [1]:
import pandas as pd
import numpy as np
import re

In [144]:
service_df = pd.read_csv("../../parsed_text_v2/parsed_rules/service_df.csv")
result_df = pd.read_csv("result_v2gpt-4-goz_commentary_v0.csv")
wrong_df = result_df[result_df["prediction"] != result_df["is_correct"]]

rules = result_df["tests_rule"].unique()
wrong_rules = wrong_df["tests_rule"].unique()
accuracy = (len(rules) - len(wrong_rules)) / len(rules)
print("accuracy", accuracy)

accuracy 0.35294117647058826


In [145]:
len(wrong_df["tests_rule"].unique())

11

In [146]:
wrong_df

,Unnamed: 0,tests_rule,invoice,result,interpreter_result,code,is_correct,explanation,prediction
4,4,65,"{'_Positionen': [{'_Faktor': 2.3, '_Behandlung...",'If service 0065 is billed for a specific area...,'If service 0065 is billed for a specific area...,\nis_invoice_invalid(Invoice) :-\n Invoice ...,True,If the service under number 0065 is included i...,False
7,7,100,"{'_Positionen': [{'_Faktor': 2.3, '_Behandlung...",False,"ERROR: -g example_invoice(X), is_invoice_inval...",\nis_invoice_invalid(Invoice) :-\n Invoice ...,False,If the service is billed more than once per to...,True
8,8,100,"{'_Positionen': [{'_Faktor': 2.3, '_Behandlung...",False,"ERROR: -g example_invoice(X), is_invoice_inval...",\nis_invoice_invalid(Invoice) :-\n Invoice ...,False,If the service is billed more than once per to...,True
10,10,110,"{'_Positionen': [{'_Faktor': 2, '_Behandlungsd...",False,"ERROR: -g example_invoice(X), is_invoice_inval...",\nis_invoice_invalid(Invoice) :-\n Invoice ...,False,The surcharge according to ServiceNumber 0110 ...,True
16,16,1020,"{'_Positionen': [{'_Faktor': 2.3, '_Behandlung...",False,"ERROR: -g example_invoice(X), is_invoice_inval...",\nis_invoice_invalid(Invoice) :-\n Invoice ...,False,In connection with the services\naccording to ...,True
19,19,2030,"{'_Positionen': [{'_Faktor': 2.3, '_Behandlung...",Error,"ERROR: -g example_invoice(X), is_invoice_inval...","\nexample_invoice(invoice([\n\t\tservice(2030,...",True,Die Leistung nach der Nummer 2030 ist je Sitzu...,False
21,21,2195,"{'_Positionen': [{'_Faktor': 2.3, '_Behandlung...",Error,"ERROR: -g example_invoice(X), is_invoice_inval...","\nexample_invoice(invoice([\n\t\tservice(2195,...",True,"Die Leistungen nach den Nummern 2180, 2190 ode...",False
24,24,2195,"{'_Positionen': [{'_Faktor': 2.3, '_Behandlung...",Error,"ERROR: -g example_invoice(X), is_invoice_inval...","\nexample_invoice(invoice([\n\t\tservice(2190,...",True,"Die Leistungen nach den Nummern 2180, 2190 und...",False
26,26,2220,"{'_Positionen': [{'_Faktor': 2.3, '_Behandlung...",Error,"ERROR: -g example_invoice(X), is_invoice_inval...","\nexample_invoice(invoice([\n\t\tservice(2220,...",True,NaN,False
29,29,2240,"{'_Positionen': [{'_Faktor': 2.3, '_Behandlung...",Error,"ERROR: -g example_invoice(X), is_invoice_inval...",\n\n\n\nexample_invoice(invoice([\n\t\tservice...,True,Die Leistung nach den Nummern 2230 oder 2240 s...,False


In [147]:
index = 7
print("gt:", wrong_df.loc[index]["is_correct"], "prediction:", wrong_df.loc[index]["prediction"])
print(wrong_df.loc[index]["code"])
print(wrong_df.loc[index]["explanation"])
print("---")
print(wrong_df.loc[index]["interpreter_result"])

gt: False prediction: True

is_invoice_invalid(Invoice) :-
    Invoice = invoice(Services, _, _, _),
    findall(T, member(service(0090, _, _, _, _, _, Justification, [T]), Services), Teeth),
    sort(Teeth, SortedTeeth),
    length(Teeth, L1),
    length(SortedTeeth, L2),
    L1 > L2,
    Justification == '',
    print('If service 0090 is billed more than once per tooth, there must be a justification.'),
    true.



example_invoice(invoice([
		service(0090, date(2012, 2, 12), 2.3, 60.0, 18.1, 'Eingehende Untersuchung zur Feststellung von Zahn-, Mund- und Kiefererkrankungen', '', [tooth(3, 1)]),
		service(0090, date(2012, 2, 12), 2.3, 60.0, 18.1, 'Eingehende Untersuchung zur Feststellung von Zahn-, Mund- und Kiefererkrankungen', '', [tooth(3, 1)])
	],
	[
		
	],
	invoice_date(date(2012, 7, 3)),
	invoice_amount(18.1)
)).
If the service is billed more than once per tooth under code 0090, this must be justified in the invoice.
---
ERROR: -g example_invoice(X), is_invoice_invalid(X).: fals

In [143]:
for idx, row in wrong_df.iterrows():
    rule_nr = row["tests_rule"]
    service = service_df[service_df["number"] == int(rule_nr)]
    print("Rule:")
    print(service["rule"].values)
    print("Translation:")
    print(row["code"])
    print("")
    print("The Translation was false for the given example invoice")
    print(f"The example invoice was: {'correct' if row['is_correct'] else 'wrong'}")
    print("The code predicted: ", 'correct' if row['prediction'] else 'wrong')
    print("The output of the code was")
    print(row["interpreter_result"])
    print("==================================================")

Rule:
['Der Zuschlag nach der Nummer 0110 ist je Behandlungstag nur einmal und nur mit dem einfachen Gebührensatz berechnungsfähig.'
 nan]
Translation:

% Helper predicate to count occurrences of service 0110 on a given date with a multiplier other than 1.0
count_service_0110([], _, Count) :- Count = 0.
count_service_0110([service(0110, Date, Multiplier, _, _, _, _, _) | Rest], Date, Count) :-
    Multiplier =\= 1.0, !,
    count_service_0110(Rest, Date, Count).
count_service_0110([service(0110, Date, _, _, _, _, _, _) | Rest], Date, Count) :-
    count_service_0110(Rest, Date, TempCount),
    Count is TempCount + 1.
count_service_0110([service(_, _, _, _, _, _, _, _) | Rest], Date, Count) :-
    count_service_0110(Rest, Date, Count).

% Predicate to check if the invoice is invalid due to rule 1
is_invoice_invalid(Invoice) :-
    Invoice = invoice(Services, _, _, _),
    member(service(0110, Date, _, _, _, _, _, _), Services),
    count_service_0110(Services, Date, Count),
    (Count >

ValueError: invalid literal for int() with base 10: 'L_Zuschläge_zu_bestimmten'

In [86]:
service_df

,number,content,section,Punktzahl,1.0-fach,2.3-fach,3.5-fach,reference,extr_ref_calls,extr_ref_prompt_tokens,extr_ref_completion_tokens,extr_ref_total_cost,parsed_references,commentary,rule,extr_rules_calls,extr_rules_prompt_tokens,extr_rules_completion_tokens,extr_rules_total_cost
0,10,Eingehende Untersuchung zur\nFeststellung von ...,A_Allgemeine_zahnärztliche_Leistungen,100.0,5.62,12.94,19.68,The text does not mention any references to ot...,1,111,11,0.00399,NaN,GOZ Nr. 0010\nEingehende Untersuchung zur Fest...,NaN,1,658,5,0.0
1,30,Aufstellung eines schriftlichen Heil-\nund Kos...,A_Allgemeine_zahnärztliche_Leistungen,200.0,11.25,25.87,39.37,The text does not mention any references to ot...,1,91,11,0.00339,NaN,A Allgemeine zahnärztliche Leistungen\nGOZ Nr....,NaN,1,638,5,0.0
2,40,Aufstellung eines schriftlichen Heil-\nund Kos...,A_Allgemeine_zahnärztliche_Leistungen,250.0,14.06,32.34,49.21,"References: Nummern 0030, 0040",1,161,12,0.00555,NaN,GOZ Nr. 0040\nAufstellung eines schriftlichen ...,Die Leistungen nach den Nummern 0030 und 0040 ...,1,708,31,0.0
3,40,Aufstellung eines schriftlichen Heil-\nund Kos...,A_Allgemeine_zahnärztliche_Leistungen,250.0,14.06,32.34,49.21,"References: Nummern 0030, 0040",1,161,12,0.00555,NaN,GOZ Nr. 0040\nAufstellung eines schriftlichen ...,NaN,1,708,31,0.0
4,50,Abformung oder Teilabformung\neines Kiefers fü...,A_Allgemeine_zahnärztliche_Leistungen,120.0,6.75,15.52,23.62,The text does not mention any references to ot...,1,96,11,0.00354,NaN,A Allgemeine zahnärztliche Leistungen\nGOZ Nr....,NaN,1,643,5,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
288,520,Zuschlag bei nichtstationärer Durch -\nführung...,L_Zuschläge_zu_bestimmten,1300.0,73.11,NaN,NaN,"References: Nummer 0520, Nummer 0500, Nummer 0...",1,186,25,0.00708,NaN,L Zuschläge zu bestimmten zahnärztlich-chirurg...,Der Zuschlag nach der Nummer 0520 ist neben de...,1,733,73,0.0
289,520,Zuschlag bei nichtstationärer Durch -\nführung...,L_Zuschläge_zu_bestimmten,1300.0,73.11,NaN,NaN,"References: Nummer 0520, Nummer 0500, Nummer 0...",1,186,25,0.00708,NaN,L Zuschläge zu bestimmten zahnärztlich-chirurg...,NaN,1,733,73,0.0
290,530,Zuschlag bei nichtstationärer Durch -\nführung...,L_Zuschläge_zu_bestimmten,2200.0,123.73,NaN,NaN,"References: Nummer 0530, Nummern 0500 bis 0520",1,182,18,0.00654,NaN,GOZ Nr. 0530\nZuschlag bei nichtstationärer Du...,Der Zuschlag nach der Nummer 0530 ist je Behan...,1,729,67,0.0
291,530,Zuschlag bei nichtstationärer Durch -\nführung...,L_Zuschläge_zu_bestimmten,2200.0,123.73,NaN,NaN,"References: Nummer 0530, Nummern 0500 bis 0520",1,182,18,0.00654,NaN,GOZ Nr. 0530\nZuschlag bei nichtstationärer Du...,Der Zuschlag nach der Nummer 0530 ist neben de...,1,729,67,0.0


In [119]:
print(wrong_df.loc[21]["explanation"])

Die Leistungen nach den Nummern 2180, 2190 oder 2195 sind neben den Leistungen nach den Nummern 2150 bis 2170 nicht berechnungsfähig.


In [25]:
result_df = pd.read_csv("result_baseline.csv")
wrong_df = result_df[result_df["prediction"] != result_df["is_correct"]]

rules = result_df["tests_rule"].unique()
wrong_rules = wrong_df["tests_rule"].unique()
accuracy = (len(rules) - len(wrong_rules)) / len(rules)
print("accuracy", accuracy)
wrong_df = result_df[result_df["prediction"] != result_df["is_correct"]]
wrong_df

accuracy 0.5882352941176471


,Unnamed: 0,tests_rule,invoice,result,is_correct,explanation,prediction,llm_response
5,5,65,"{'_Positionen': [{'_Faktor': 2.3, '_Behandlung...",Yes,False,If the service under number 0065 is included i...,True,Reasoning: The rule provided is about a specif...
10,10,110,"{'_Positionen': [{'_Faktor': 2, '_Behandlungsd...",Yes,False,The surcharge according to ServiceNumber 0110 ...,True,Reasoning: The rule states that the surcharge ...
19,19,2030,"{'_Positionen': [{'_Faktor': 2.3, '_Behandlung...",No,True,Die Leistung nach der Nummer 2030 ist je Sitzu...,False,Reasoning: The rule states that the service wi...
26,26,2220,"{'_Positionen': [{'_Faktor': 2.3, '_Behandlung...",No,True,Neben den Leistungen nach den Nummern 2200 bis...,False,Reasoning: The rule is about dental services r...
31,31,2400,"{'_Positionen': [{'_Faktor': 2.3, '_Behandlung...",No,True,Die Leistung nach der Nummer 2400 ist je Wurze...,False,Reasoning: The rule states that the service wi...
32,32,2400,"{'_Positionen': [{'_Faktor': 2.3, '_Behandlung...",Unclear,True,Die Leistung nach der Nummer 2400 ist je Wurze...,False,Reasoning: The rule states that the service wi...
35,35,2410,"{'_Positionen': [{'_Faktor': 2.3, '_Behandlung...",No,True,Die Leistung nach der Nummer 2410 ist für dens...,False,Reasoning: The rule states that the service wi...
38,38,L_Zuschläge_zu_bestimmten,{'_Positionen': [{'_Behandlungsdatum': '2012-0...,No,True,Die Zuschläge nach den Nummern 0500 bis 0530 s...,False,Reasoning: The invoice is not correct accordin...
39,39,L_Zuschläge_zu_bestimmten,{'_Positionen': [{'_Behandlungsdatum': '2012-0...,No,True,Die Zuschläge nach den Nummern 0500 bis 0530 s...,False,Reasoning: The rule states that the surcharges...


In [202]:
import numpy as np

accuracies = []
n_correct_invoices = []
n_wrong_invoices = []
n_correct_rules = []
n_wrong_rules = []


for i in range(5):
    df = pd.read_csv(f"result_gpt-4-1106-preview_no_rule_extraction_v{i}.csv")
    wrong_df = df[df["prediction"] != df["is_correct"]]

    rules = df["tests_rule"].unique()
    wrong_rules = wrong_df["tests_rule"].unique()
    accuracy = (len(rules) - len(wrong_rules)) / len(rules)
    accuracies.append(accuracy)
    print("accuracy", accuracy)
    n_correct_invoices.append(len(df[df["is_correct"] == df["prediction"]]))
    n_wrong_invoices.append(len(wrong_df))
    n_correct_rules.append(len(rules) - len(wrong_rules))
    n_wrong_rules.append(len(wrong_rules))
    
print("mean accuracy", sum(accuracies) / len(accuracies), "std", np.std(accuracies))
print("mean n_correct_invoices", sum(n_correct_invoices) / len(n_correct_invoices), "std", np.std(n_correct_invoices))
print("mean n_wrong_invoices", sum(n_wrong_invoices) / len(n_wrong_invoices), "std", np.std(n_wrong_invoices))
print("mean n_correct_rules", sum(n_correct_rules) / len(n_correct_rules), "std", np.std(n_correct_rules))
print("mean n_wrong_rules", sum(n_wrong_rules) / len(n_wrong_rules), "std", np.std(n_wrong_rules))

    

accuracy 0.5882352941176471
accuracy 0.5882352941176471
accuracy 0.7058823529411765
accuracy 0.5882352941176471
accuracy 0.47058823529411764
mean accuracy 0.5882352941176471 std 0.07440653318043247
mean n_correct_invoices 35.6 std 1.9595917942265424
mean n_wrong_invoices 9.4 std 1.9595917942265424
mean n_correct_rules 10.0 std 1.2649110640673518
mean n_wrong_rules 7.0 std 1.2649110640673518


In [7]:
path = "../../results/gpt-4_v"

dfs = []
sums = []
means = []
for i in range(5):
    section_df = pd.read_csv(f"{path}{i}/section_df.csv")
    text_df = pd.read_csv(f"{path}{i}/text_df.csv")
    service_df = pd.read_csv(f"{path}{i}/service_df.csv")
    dfs.append((section_df, text_df, service_df))
    sum_tokens = section_df["completion_tokens"].sum() + text_df["completion_tokens"].sum() + service_df["completion_tokens"].sum()
    mean_tokens = np.mean([section_df["completion_tokens"].dropna().mean(), text_df["completion_tokens"].dropna().mean(), service_df["completion_tokens"].dropna().mean()])
    sums.append(sum_tokens)
    means.append(mean_tokens)
    

In [8]:
sums

[24979.0, 23624.0, 20460.0, 21177.0, 11268.0]

In [9]:
means

[870.9242424242424,
 751.849358974359,
 670.2393162393163,
 717.7321428571428,
 452.2238095238095]

In [10]:
np.mean(sums)

20301.6

In [2]:
np.mean(means)

NameError: name 'means' is not defined

In [159]:
text_df = pd.read_csv("../../results/v2/v2gpt-4_v0/text_df.csv")
section_df = pd.read_csv("../../results/v2/v2gpt-4_v0/section_df.csv")
service_df = pd.read_csv("../../results/v2/v2gpt-4_v0/service_df.csv")

In [160]:
text_df

,Unnamed: 0,section,subsection,content,reference,extr_ref_calls,extr_ref_prompt_tokens,extr_ref_completion_tokens,extr_ref_total_cost,section_number,...,extr_rules_total_cost,llm_response,query,llm_inputs,llm_calls,prompt_tokens,completion_tokens,total_cost,parsed_code,correction_count
0,0,NaN,NaN,Erste Verordnung zur Änderung der Gebührenordn...,References: § 15,1,240,5,0.00750,NaN,...,0.02376,NaN,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,§ 1 Anwendungsbereich,(1),Die Vergütungen für die beruflichen Leistungen...,The text does not mention any references to ot...,1,102,11,0.00372,1.0,...,0.01962,NaN,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,§ 1 Anwendungsbereich,(2),Vergütungen darf der Zahnarzt nur für Leistung...,The text does not mention any references to ot...,1,170,11,0.00576,1.0,...,0.02166,NaN,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,§ 2 Abweichende Vereinbarung,(1),Durch Vereinbarung zwischen Zahnarzt und Zahlu...,References: §5 (1),1,191,7,0.00615,2.0,...,0.03540,---\nRule 1:\nThoughts: This rule states that ...,NaN,Given this rules from the german GOZ:\n---\nRu...,1.0,1942.0,876.0,0.11082,\n,0.0
4,4,§ 2 Abweichende Vereinbarung,(1),Durch Vereinbarung zwischen Zahnarzt und Zahlu...,References: §5 (1),1,191,7,0.00615,2.0,...,0.03540,NaN,n,NaN,NaN,NaN,NaN,NaN,\n,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62,62,§ 10 Fälligkeit und Abrechnung der Vergütung; ...,(6),Die Übermittlung von Daten an einen Dritten zu...,The text does not mention any references to ot...,1,145,11,0.00501,10.0,...,0.02091,NaN,n,NaN,NaN,NaN,NaN,NaN,NaN,NaN
63,63,§ 11 Übergangsvorschrift,NaN,Die Gebührenordnung für Zahnärzte in der vor d...,The text does not mention any references to ot...,1,400,11,0.01266,11.0,...,0.04314,---\nRule 1:\nThoughts: This rule states that ...,NaN,Given this rules from the german GOZ:\n---\nRu...,1.0,1444.0,555.0,0.07662,\n,0.0
64,64,§ 11 Übergangsvorschrift,NaN,Die Gebührenordnung für Zahnärzte in der vor d...,The text does not mention any references to ot...,1,400,11,0.01266,11.0,...,0.04314,NaN,n,NaN,NaN,NaN,NaN,NaN,\n,0.0
65,65,§ 11 Übergangsvorschrift,NaN,Die Gebührenordnung für Zahnärzte in der vor d...,The text does not mention any references to ot...,1,400,11,0.01266,11.0,...,0.04314,NaN,n,NaN,NaN,NaN,NaN,NaN,\n,0.0


In [161]:
col = "rule"
num_rules = len(text_df[col].dropna()) + len(section_df[col].dropna()) + len(service_df[col].dropna())
print(col, ":", num_rules)

col = "correction_count"
sum = text_df[col].sum() + section_df[col].sum() + service_df[col].sum()
print(col, ":", sum, sum/ num_rules)

col = "total_cost"
sum = text_df[col].sum() + section_df[col].sum() + service_df[col].sum()
print(col, ":", sum, sum/ num_rules)
col = "prompt_tokens"
sum = text_df[col].sum() + section_df[col].sum() + service_df[col].sum()
print(col, ":", sum, sum/ num_rules)
col = "completion_tokens"
sum = text_df[col].sum() + section_df[col].sum() + service_df[col].sum()
print(col, ":", sum, sum/ num_rules)
col = "llm_calls"
sum = text_df[col].sum() + section_df[col].sum() + service_df[col].sum()
print(col, ":", sum, sum/ num_rules)

rule : 144
correction_count : 0.0 0.0
total_cost : 1.9455899999999997 0.013511041666666664
prompt_tokens : 29521.0 205.00694444444446
completion_tokens : 17666.0 122.68055555555556
llm_calls : 20.0 0.1388888888888889


In [153]:
col = "extr_rules_total_cost"
sum = text_df[col].sum() + section_df[col].sum() + service_df[col].sum()
print(col, ":", sum, sum/ num_rules)
col = "extr_rules_prompt_tokens"
sum = text_df[col].sum() + section_df[col].sum() + service_df[col].sum()
print(col, ":", sum, sum/ num_rules)
col = "extr_rules_completion_tokens"
sum = text_df[col].sum() + section_df[col].sum() + service_df[col].sum()
print(col, ":", sum, sum/ num_rules)
col = "extr_rules_calls"
sum = text_df[col].sum() + section_df[col].sum() + service_df[col].sum()
print(col, ":", sum, sum/ num_rules)

extr_rules_total_cost : 0.0 0.0
extr_rules_prompt_tokens : 312521 2155.3172413793104
extr_rules_completion_tokens : 38838 267.848275862069
extr_rules_calls : 390 2.689655172413793


In [152]:
col = "extr_ref_total_cost"
sum = text_df[col].sum() + section_df[col].sum() + service_df[col].sum()
print(col, ":", sum, sum/ num_rules)
col = "extr_ref_prompt_tokens"
sum = text_df[col].sum() + section_df[col].sum() + service_df[col].sum()
print(col, ":", sum, sum/ num_rules)
col = "extr_ref_completion_tokens"
sum = text_df[col].sum() + section_df[col].sum() + service_df[col].sum()
print(col, ":", sum, sum/ num_rules)
col = "extr_ref_calls"
sum = text_df[col].sum() + section_df[col].sum() + service_df[col].sum()
print(col, ":", sum, sum/ num_rules)

extr_ref_total_cost : 2.476799999999987 0.017081379310344737
extr_ref_prompt_tokens : 70042 483.048275862069
extr_ref_completion_tokens : 6259 43.16551724137931
extr_ref_calls : 390 2.689655172413793


144

In [162]:
np.mean([2/6, 2/7, 1/5, 2/6, 1/5])

0.2704761904761904

In [163]:
np.mean([4/6, 5/7, 4/5, 4/6, 4/5])

0.7295238095238096

In [164]:
np.mean([5/10, 4/10, 3/10, 3/10, 2/8])

0.35

In [28]:
np.std([2/17, 1/17, 1/17, 2/17, 2/17])

0.02881752638568445

In [1]:

from datetime import datetime
from typing import Dict, Any

import pandas as pd


def json_to_prolog(invoice_data: Dict[str, Any], service_df: pd.DataFrame) -> str:
    # Extract and format the invoice date.
    invoice_date = datetime.strptime(invoice_data["_Rechnungsdatum"], '%Y-%m-%d')
    prolog_invoice_date = f"date({invoice_date.year}, {invoice_date.month}, {invoice_date.day})"

    # Extract the invoice amount.
    invoice_amount = invoice_data["_Rechnungsbetrag"]

    # Lists to hold the Prolog representation of services and material costs.
    services_prolog_list = []
    material_costs_prolog_list = []

    # Process each item in the invoice.
    for item in invoice_data["_Positionen"]:
        try:
            item_date = datetime.strptime(item["_Behandlungsdatum"], '%Y-%m-%d')
            prolog_item_date = f"date({item_date.year}, {item_date.month}, {item_date.day})"
        except:
            prolog_item_date = "date('', '', '')"

        item_multiplier = item["_Faktor"]
        item_count = item["_Anzahl"]
        item_charge = item["_Betrag"]
        item_description = item["_Leistungsbeschreibung"]
        item_number = item["_Nr"]
        item_justification = item.get("_Begründung", "")

        # Convert tooth areas to Prolog list format.
        tooth_areas = ", ".join([f"tooth({str(area)[0]}, {str(area)[1]})" for area in item.get("_Zahn", [])])
        prolog_tooth_list = f"[{tooth_areas}]"

        # Depending on the type, create a service or a material cost entry.
        if item["_Typ"] == "Service":
            point_score = service_df[service_df["number"] == int(item_number)]["Punktzahl"].values[0]
            # if item count > 0 write the service count times. When the number of teeth == count,
            # then write the service for each tooth.
            if 1 < item_count == len(item.get("_Zahn", [])):
                prolog_teeth = [f"[tooth({str(area)[0]}, {str(area)[1]})]" for area in item.get("_Zahn", [])]
            else:
                prolog_teeth = [prolog_tooth_list]

            for i, teeth in zip(range(item_count), prolog_teeth):
                charge = item_charge / len(prolog_teeth)
                service_prolog = f"service({item_number}, {prolog_item_date}, {item_multiplier}, {point_score}, {charge}, \'{item_description}\', \'{item_justification}\', {teeth})"
                services_prolog_list.append(service_prolog)

        elif item["_Typ"] == "Materialkosten":
            material_cost_prolog = f"material_cost(\'{item_number}\', {prolog_item_date}, {item_multiplier}, {item_count}, {item_charge}, \'{item_description}\', \'{item_justification}\', {prolog_tooth_list})"
            material_costs_prolog_list.append(material_cost_prolog)

    # Convert the lists to strings.
    list_of_services = ",\n\t\t".join(services_prolog_list)

    list_of_material_costs = ",\n\t\t".join(material_costs_prolog_list)
    list_of_material_costs = f",\n\t[\n\t\t{list_of_material_costs}\n\t]"


    # Combine everything into the final Prolog representation.
    prolog_representation = f"invoice([\n\t\t{list_of_services}\n\t]{list_of_material_costs},\n\tinvoice_date({prolog_invoice_date}),\n\tinvoice_amount({invoice_amount})\n)."

    return prolog_representation

In [2]:
invoice_dirk_dental = {
        "_Positionen": [
            {
                "_Behandlungsdatum": "2016-09-02",
                "_Faktor": 1.0,
                "_Anzahl": 1,
                "_Betrag": 11.25,
                "_Leistungsbeschreibung": "Erstellung eines Mundhygienestatus und eigenhende Unterweisung zur Vorbeugung gegen Karies und parodontale Erkrankungen, Dauer mindestens 25 Minuten",
                "_Nr": "1000",
                "_Typ": "Service"
            },
            {
                "_Behandlungsdatum": "2016-09-02",
                "_Faktor": 2.0,
                "_Anzahl": 28,
                "_Betrag": 88.20,
                "_Leistungsbeschreibung": "Professionelle Zahnreinigung (Zahn: 17,16,15,14,13,12,11,21,22,23,24,25,26,27,37,36,35,34,33,32,31,41,42,43,44,45,46,47)",
                "_Zahn": [17,16,15,14,13,12,11,21,22,23,24,25,26,27,37,36,35,34,33,32,31,41,42,43,44,45,46,47],
                "_Nr": "1040",
                "_Typ": "Service"
            },
            {
                "_Behandlungsdatum": "2016-09-06",
                "_Faktor": 3.5,
                "_Anzahl": 1,
                "_Betrag": 151.57,
                "_Leistungsbeschreibung": "Präparieren einer Kavität und Restauration mit Kompositmaterialien, in Adhäsivtechnik (Konditionieren), mehr als dreiflächig, ggf. einschließlich Polieren, ggf. einschließlich Verwendung von Inserts",
                "_Begründung": "Überdurchscnittlicher Schwierigkeitsgrad und Zeitaufwand wegen erschwertem Anlegen von Matrizen/Bändern durch Zahnengstand und tiefer, sehr schwer einsehbarer Approximalkavität",
                "_Nr": "2120",
                "_Typ": "Service"
            }
        ],
        "_Kassenanteil": {
            "_KassenanteilSumme": -54.47
        },
        "_Rechnungsnummer": "92545-16",
        "_Rechnungsdatum": "2016-11-11",
        "_Rechnungsbetrag": 251.02,
        "_Waehrung": "EUR"
    }

In [3]:
service_df = pd.read_csv("../../parsed_text/de/service_df.csv")

In [4]:
print(json_to_prolog(invoice_dirk_dental, service_df))

invoice([
		service(1000, date(2016, 9, 2), 1.0, 200.0, 11.25, 'Erstellung eines Mundhygienestatus und eigenhende Unterweisung zur Vorbeugung gegen Karies und parodontale Erkrankungen, Dauer mindestens 25 Minuten', '', []),
		service(1040, date(2016, 9, 2), 2.0, 28.0, 3.15, 'Professionelle Zahnreinigung (Zahn: 17,16,15,14,13,12,11,21,22,23,24,25,26,27,37,36,35,34,33,32,31,41,42,43,44,45,46,47)', '', [tooth(1, 7)]),
		service(1040, date(2016, 9, 2), 2.0, 28.0, 3.15, 'Professionelle Zahnreinigung (Zahn: 17,16,15,14,13,12,11,21,22,23,24,25,26,27,37,36,35,34,33,32,31,41,42,43,44,45,46,47)', '', [tooth(1, 6)]),
		service(1040, date(2016, 9, 2), 2.0, 28.0, 3.15, 'Professionelle Zahnreinigung (Zahn: 17,16,15,14,13,12,11,21,22,23,24,25,26,27,37,36,35,34,33,32,31,41,42,43,44,45,46,47)', '', [tooth(1, 5)]),
		service(1040, date(2016, 9, 2), 2.0, 28.0, 3.15, 'Professionelle Zahnreinigung (Zahn: 17,16,15,14,13,12,11,21,22,23,24,25,26,27,37,36,35,34,33,32,31,41,42,43,44,45,46,47)', '', [tooth(1, 4)